In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np

import tensorflow as tf

!pip install -q tensorflow-hub
!pip install -q tfds-nightly
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

Version:  2.1.0
Eager mode:  True
Hub version:  0.7.0
GPU is NOT AVAILABLE


In [2]:
# Split the training set into 60% and 40%, so we'll end up with 15,000 examples
# for training, 10,000 examples for validation and 25,000 examples for testing.
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

Shuffling and writing examples to /Users/cesarsalcedo/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteMON8D1/imdb_reviews-train.tfrecord


Shuffling and writing examples to /Users/cesarsalcedo/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteMON8D1/imdb_reviews-test.tfrecord


Shuffling and writing examples to /Users/cesarsalcedo/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteMON8D1/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /Users/cesarsalcedo/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [4]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))

In [27]:
embeddings = [
    "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1",
    "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim-with-oov/1",
    "https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1",
    "https://tfhub.dev/google/tf2-preview/nnlm-en-dim128/1"
]

hub_layer = hub.KerasLayer(embeddings[3], input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 128), dtype=float32, numpy=
array([[ 9.01966274e-01, -4.83913347e-03,  1.17907055e-01,
         3.81319046e-01,  6.57222793e-02, -3.01581532e-01,
         8.90584365e-02, -2.69034863e-01, -8.51345584e-02,
         1.08877886e-02, -6.66372627e-02, -3.73063087e-01,
        -2.76447266e-01, -1.87254980e-01,  5.67507632e-02,
         9.09779966e-02, -6.24961555e-02, -3.28687276e-03,
        -3.08512092e-01,  3.78482223e-01,  7.62880966e-02,
         1.43733576e-01, -1.12897493e-01,  9.59761534e-03,
        -2.38938913e-01,  2.93743908e-02,  7.28663057e-02,
        -2.48727947e-02, -8.16893280e-02,  6.68320432e-02,
        -5.62225394e-02,  2.47078985e-01,  1.17681175e-01,
         3.17581035e-02,  2.65932620e-01, -1.37706831e-01,
        -1.50708258e-01, -1.63614675e-01, -1.51269153e-01,
         2.34616160e-01, -9.12236273e-02, -4.22684886e-02,
        -1.01224177e-01, -2.12229744e-01,  6.74503446e-02,
         1.85163647e-01,  3.62982228e-02, -3.50210071e-01,
      

In [28]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_5 (KerasLayer)   (None, 128)               124642688 
_________________________________________________________________
dense_10 (Dense)             (None, 16)                2064      
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 17        
Total params: 124,644,769
Trainable params: 124,644,769
Non-trainable params: 0
_________________________________________________________________


In [29]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [30]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=5,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/5
30/30 [==============================] - 52s 2s/step - loss: 0.6016 - accuracy: 0.5898 - val_loss: 0.5116 - val_accuracy: 0.7002
Epoch 2/5
30/30 [==============================] - 50s 2s/step - loss: 0.4045 - accuracy: 0.8171 - val_loss: 0.3749 - val_accuracy: 0.8418
Epoch 3/5
30/30 [==============================] - 50s 2s/step - loss: 0.2576 - accuracy: 0.9005 - val_loss: 0.3183 - val_accuracy: 0.8543
Epoch 4/5
30/30 [==============================] - 52s 2s/step - loss: 0.1692 - accuracy: 0.9424 - val_loss: 0.3012 - val_accuracy: 0.8756
Epoch 5/5
30/30 [==============================] - 53s 2s/step - loss: 0.1088 - accuracy: 0.9699 - val_loss: 0.2994 - val_accuracy: 0.8737


In [32]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

loss: 0.319
accuracy: 0.858
